In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [13]:
from llama_index.legacy import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("aboutGoogle").load_data()

In [7]:
import sys
print(sys.executable)
!pip list

c:\Users\poora\OneDrive\Desktop\projects\RAG\env\Scripts\python.exe
Package                                 Version
--------------------------------------- -----------
aiohttp                                 3.9.3
aiosignal                               1.3.1
annotated-types                         0.6.0
anyio                                   4.3.0
asttokens                               2.4.1
async-timeout                           4.0.3
attrs                                   23.2.0
beautifulsoup4                          4.12.3
certifi                                 2024.2.2
charset-normalizer                      3.3.2
click                                   8.1.7
colorama                                0.4.6
comm                                    0.2.2
dataclasses-json                        0.6.4
debugpy                                 1.8.1
decorator                               5.1.1
Deprecated                              1.2.14
dirtyjson                               1.0.


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
documents

[Document(id_='3bf7f3dc-c4db-4d72-a796-c2bd3fc755b6', embedding=None, metadata={'file_path': 'aboutSwiggy\\swiggy_info.txt', 'file_name': 'swiggy_info.txt', 'file_type': 'text/plain', 'file_size': 18538, 'creation_date': '2024-04-16', 'last_modified_date': '2024-04-16', 'last_accessed_date': '2024-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Swiggy is India’s largest online food item ordering & delivery chain, it also tops the chart of India’s Unicorn startup lists. It’s a Bangalore-based startup started in 2014, and as of now, it’s expanded to more than 100 Indian cities. Swiggy propelled quick pick-and-drop food delivery applications to make the life of people simpler. It gives a single window to request from an extensive variety of restaur

In [15]:
index = VectorStoreIndex.from_documents(documents,show_progress=True)

c:\Users\poora\OneDrive\Desktop\projects\RAG\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 5/5 [00:01<00:00,  3.61it/s]


In [16]:
index

In [12]:
query_engine = index.as_query_engine()

In [13]:
query_engine

In [5]:
from llama_index.core import Settings

In [6]:
from llama_index.legacy import  StorageContext

In [7]:
from llama_index.legacy import load_index_from_storage

In [ ]:
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader("aboutGoogle").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
    
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)
    
query_engine = index.as_query_engine() # print single response
response = query_engine.query("Can you give a little information about Google's culture and values")
print(response)

In [ ]:
# top k retrieved responses

from llama_index.legacy.retrievers import VectorIndexRetriever
from llama_index.legacy.query_engine import RetrieverQueryEngine
from llama_index.legacy.postprocessor import SimilarityPostprocessor

retriever = VectorIndexRetriever(index=index,similarity_top_k=5)
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.80)
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[postprocessor])

response = query_engine.query("what challenges a fresher might face at Google?")
from llama_index.legacy.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

In [ ]:
chat_engine = index.as_chat_engine()
streaming_response = chat_engine.stream_chat("what challenges a fresher might face at Google?")
for token in streaming_response.response_gen:
    print(token, end="")